In [2]:
def parse_input(file):
    with open(file, 'r') as file_in:
        data = file_in.read().split('\n\n')
    
    workflows = {}
    for wf in data[0].splitlines():
        wf_name, wf_rules = wf.split('{')
        workflows[wf_name] = {}
        for rule in wf_rules.replace('}', '').split(','):
            rule_split = rule.split(':')
            if len(rule_split) == 2:
                key, val = rule_split
                workflows[wf_name][key] = val
            elif len(rule_split) == 1:
                final = rule_split[0]
                if final in ['A', 'R']:
                    workflows[wf_name][final] = None
                else:
                    workflows[wf_name]['True'] = final
                    
    ratings = []
    for part in data[1].splitlines():
        row = part.replace('{', '').replace('}', '').split(',')
        ratings.append(tuple(row))
        

    return workflows, tuple(ratings)

In [3]:
def validate_part(part_ratings, workflows):

    for statement in part_ratings:
        exec(statement)

    decision = 'in'
    while decision not in ['A', 'R']:
        for wf_rule in workflows[decision]:
            if wf_rule in ['A', 'R']:
                decision = wf_rule
                break
            elif eval(wf_rule):
                decision =  workflows[decision][wf_rule]
                break
            
    return decision

In [4]:
def sum_ratings_accepted(all_ratings, workflows):
    total_ratings = 0
    for part_ratings in all_ratings:
        if validate_part(part_ratings, workflows) == 'A':
            total_ratings += sum([int(rat.split('=')[1]) for rat in part_ratings])
            
    return total_ratings

In [5]:
def test(workflows, possible_paths=None):
    
    if possible_paths is None:
        current_paths = []
    else:
        current_paths = possible_paths
    
    for k, v in workflows[path[-1]].items():
        if k == 'R' or v == 'R':
            break
        elif k == 'A' or v == 'A':
            path.append('A')
            return path
        else:
            path.append(v)
            test(workflows, path)
    return path

workflows, all_ratings = parse_input("calibration.txt")
sum_ratings_accepted(all_ratings, workflows)
test(workflows, path = ['in'])

['in', 'px', 'qkq', 'A', 'A', 'qqz', 'qs', 'A', 'hdj', 'A']

In [7]:
workflows

{'px': {'a<2006': 'qkq', 'm>2090': 'A', 'True': 'rfg'},
 'pv': {'a>1716': 'R', 'A': None},
 'lnx': {'m>1548': 'A', 'A': None},
 'rfg': {'s<537': 'gd', 'x>2440': 'R', 'A': None},
 'qs': {'s>3448': 'A', 'True': 'lnx'},
 'qkq': {'x<1416': 'A', 'True': 'crn'},
 'crn': {'x>2662': 'A', 'R': None},
 'in': {'s<1351': 'px', 'True': 'qqz'},
 'qqz': {'s>2770': 'qs', 'm<1801': 'hdj', 'R': None},
 'gd': {'a>3333': 'R', 'R': None},
 'hdj': {'m>838': 'A', 'True': 'pv'}}

In [8]:
from collections import deque

In [27]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if graph[start][node] not in path:
            newpaths = find_all_paths(graph, graph[start][node], end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

# Search for all paths leading to 'A'
find_all_paths(workflows, 'in', 'A')

[['in', 'px', 'qkq', 'A'],
 ['in', 'px', 'qkq', 'crn', 'A'],
 ['in', 'px', 'A'],
 ['in', 'qqz', 'qs', 'A'],
 ['in', 'qqz', 'qs', 'lnx', 'A'],
 ['in', 'qqz', 'hdj', 'A']]

In [22]:
def test(workflows, possible_paths=None):
    
    if possible_paths is None:
        current_paths = []
    else:
        current_paths = possible_paths
    
    for k, v in workflows[path[-1]].items():
        if k == 'R' or v == 'R':
            break
        elif k == 'A' or v == 'A':
            path.append('A')
            return path
        else:
            path.append(v)
            test(workflows, path)
    return path

workflows, all_ratings = parse_input("calibration.txt")
sum_ratings_accepted(all_ratings, workflows)
test(workflows, path = ['in'])

{'px': {'a<2006': 'qkq', 'm>2090': 'A', 'True': 'rfg'},
 'pv': {'a>1716': 'R', 'A': None},
 'lnx': {'m>1548': 'A', 'A': None},
 'rfg': {'s<537': 'gd', 'x>2440': 'R', 'A': None},
 'qs': {'s>3448': 'A', 'True': 'lnx'},
 'qkq': {'x<1416': 'A', 'True': 'crn'},
 'crn': {'x>2662': 'A', 'R': None},
 'in': {'s<1351': 'px', 'True': 'qqz'},
 'qqz': {'s>2770': 'qs', 'm<1801': 'hdj', 'R': None},
 'gd': {'a>3333': 'R', 'R': None},
 'hdj': {'m>838': 'A', 'True': 'pv'}}

In [91]:
def count_combinations_inequality(rule):
    match = re.search('[xmas]([<>])(\d+)', rule)
    sign = match.group(1)
    value = int(match.group(2))
    
    if sign == '<':
        n_true = value - 1
        n_false = 4000 - n_true
    else:
        n_true = 4000 - value
        n_false = 4000 - n_true
        
    return n_true, n_false

In [153]:
def workflow_to_combinations(workflows):
    all_wf_combinations = {}
    for wf_name in workflows:
        n_alt_cum = 1
        wf_combinations = {}
        for key, value in workflows[wf_name].items():
            if '<' in key or '>' in key:
                n_combi, n_alt = count_combinations_inequality(key)
                n_combi *= n_alt_cum
                n_alt_cum *= n_alt
                wf_combinations[value] = n_combi
            elif key == 'True':
                wf_combinations[value] = n_alt_cum
            elif key == 'A':
                wf_combinations['A'] = n_alt_cum
            elif key == 'R' and value is None:
                wf_combinations['R'] = n_alt_cum

        if len(wf_combinations.keys()) == 1 and ('A' in wf_combinations.keys() or 'R' in wf_combinations.keys()):
            letter = list(wf_combinations.keys())[0]
            wf_combinations[letter] = 1

        all_wf_combinations[wf_name] = wf_combinations
        
    return all_wf_combinations

wf_to_combinations = workflow_to_combinations(workflows)
wf_to_combinations

{'px': {'qkq': 2005, 'A': 3810450, 'rfg': 4169550},
 'pv': {'R': 2284, 'A': 1716},
 'lnx': {'A': 1},
 'rfg': {'gd': 536, 'R': 5403840, 'A': 8452160},
 'qs': {'A': 552, 'lnx': 3448},
 'qkq': {'A': 1415, 'crn': 2585},
 'crn': {'A': 1338, 'R': 2662},
 'in': {'px': 1350, 'qqz': 2650},
 'qqz': {'qs': 1230, 'hdj': 4986000, 'R': 6094000},
 'gd': {'R': 1},
 'hdj': {'A': 3162, 'pv': 838}}